In [1]:
import numpy as np

import importlib

import findatree.io as io
import findatree.transformations as transformations
import findatree.interactive as interactive
import findatree.geo_to_image as geo_to_image
import findatree.segmentation as segmentation
import findatree.object_properties as obj_props

from bokeh.plotting import show, save
from bokeh.io import output_notebook, output_file
# output_file(r'C:\Users\flori\Documents\prof\lwf\plots\220628\tnr11878.html')
output_notebook()

Loading BokehJS ...

### Load channels

In [5]:
importlib.reload(geo_to_image)

dir_names=[]
dir_names.extend([r'C:\Data\lwf\DSM_2020'])
dir_names.extend([r'C:\Data\lwf\DTM'])
dir_names.extend([r'C:\Data\lwf\Orthophotos_2020'])
dir_names.extend([r'C:\Data\lwf\CrownDelineation_2020'])

params_channels = {
    'tnr': 11878,
    'px_width_reproject': 0.2,
    'downscale': 0,
}

channels, params_channels = geo_to_image.channels_load(dir_names, params_channels)
io.channels_to_hdf5(channels, params_channels)

-----------
Parameters:
  affine                        : [[ 2.00000000e-01  0.00000000e+00  6.58480636e+05]
 [ 0.00000000e+00 -2.00000000e-01  5.45849486e+06]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
  crs                           : EPSG:25832
  date_time                     : 220713-144756
  downscale                     : 0
  path_dsm                      : C:\Data\lwf\DSM_2020\Tnr_11878_P4M_D20200825_DSM_GCG.tif
  path_dtm                      : C:\Data\lwf\DTM\tnr_11878_dtm.tif
  path_ortho                    : C:\Data\lwf\Orthophotos_2020\Tnr_11878_2020_Ortho.tif
  path_shapes                   : C:\Data\lwf\CrownDelineation_2020\Tnr_11878_Kr.shp
  px_width                      : 0.2
  px_width_reproject            : 0.2
  shape                         : (724, 676)
  tnr                           : 11878


### Load ground-truth crowns

In [6]:
importlib.reload(io)

crowns_human, params_crowns_human = io.load_shapefile(dir_names, params_channels)
io.crowns_to_hdf5(crowns_human, params_crowns_human)

-----------
Parameters:
  affine                        : [[ 2.00000000e-01  0.00000000e+00  6.58480636e+05]
 [ 0.00000000e+00 -2.00000000e-01  5.45849486e+06]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
  date_time_polygons            : 220713-144802
  date_time_terrestrial         : 220713-144802
  number_crowns                 : 9
  origin                        : human
  path_shapes                   : C:\Data\lwf\CrownDelineation_2020\Tnr_11878_Kr.shp
  tnr                           : 11878


### Segmentation

#### (1) Parametrized Segmentation

In [29]:
importlib.reload(segmentation)
importlib.reload(io)

params_crowns_water = {
    # 'thresh_channel': 'chm',
    'thresh_blur': True,
    # 'thresh_width':30,
    # 'thresh_downscale': 2,
    # 'water_channel':'chm',
    # 'water_hole_min_area': 0.2,
}

crowns_water, params_crowns_water = segmentation.watershed(
    channels,
    params_channels,
    params_crowns_water,
)
io.crowns_to_hdf5(crowns_water, params_crowns_water)

-----------
Parameters:
  affine                        : [[ 2.00000000e-01  0.00000000e+00  6.58480636e+05]
 [ 0.00000000e+00 -2.00000000e-01  5.45849486e+06]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
  date_time_polygons            : 220713-154250
  number_crowns                 : 510
  origin                        : water
  thresh_blur                   : True
  thresh_channel                : light
  thresh_downscale              : 1
  thresh_global_chm             : 5
  thresh_global_ndvi            : 0.4
  thresh_px_width               : 0.4
  thresh_shape                  : (362, 338)
  thresh_width                  : 30
  tnr                           : 11878
  water_channel                 : light
  water_downscale               : 0
  water_hole_min_area           : 0.0
  water_label_min_area          : 0.2
  water_peak_dist               : 1.2
  water_px_width                : 0.2
  water_shape                   : (724, 676)


#### Test of burning polygons into image -> transformations.polygon_to_labelimage()

In [ ]:
import shapely.geometry

img_burn = np.zeros(img.shape, dtype=np.uint32)


shapes = [ 
    (
        shapely.geometry.mapping(shapely.geometry.Polygon(val['polygon'])),
        key,
    ) 
    for key, val in crowns_water.items()]


rasterio.features.rasterize(shapes, out = img_burn)

segments['burn'] = img_burn

#### (2) Plot segmentation results

In [ ]:
importlib.reload(interactive)

plt = interactive.Plotter()
plt.width = 600
plt.channels_downscale = 1

plt.add_channels(channels, params_channels)
plt.add_segments(segments)

# plt.figures_add_rgb()
plt.figures_add_gray('burn')
plt.figures_add_gray('labels')

plt.togglers_add_bounds()
# plt.togglers_add_crowns(crowns_water, params_crowns_water)
# plt.togglers_add_crowns(crowns_human, params_crowns_human)

layout = plt.create_layout()

In [ ]:
show(layout)
# save(layout, r"C:\Users\flori\Documents\prof\lwf\plots\220706\tnr" +  params_channels['tnr'] + "_blur.html")

### Object properties
#### (1) Extract object properties

In [ ]:
# importlib.reload(obj_props)
# props, names = obj_props.labels_to_props_all(cs_segment['labels'], cs, params_cs, include_labels=None)

In [ ]:
# # print(names)

# xlim = [5e-3, 1]

# # Filter
# name = 'perc75_chm'
# positives = props[:, names.index(name)]
# positives = positives > 5

# select = [
#     'perc75_blue',
#     'perc75_green',
#     'perc75_red',
#     'perc75_re',
#     'perc75_nir',
#     ]

# # select = [
# #     'mean_upperl_blue',
# #     'mean_upperl_green',
# #     'mean_upperl_red',
# #     'mean_upperl_re',
# #     'mean_upperl_nir',
# #     ]

# # bins = np.linspace(0.5,1,70)
# bins= 50


# f, ax = plt.subplots(figsize=[4,3])
# for name in select:
#     _ = ax.hist(
#         props[:, names.index(name)][positives],
#         bins=bins,
#         histtype='step',
#         label=name,
#         )

# _ = ax.legend(loc='upper left',bbox_to_anchor=(1,1))
# # ax.set_xlim(xlim)
# # ax.set_xscale('log')

In [ ]:
# # print(names)

# # Filter
# name = 'perc75_chm'
# positives = props[:, names.index(name)]
# positives = positives > 5

# # Variables
# name1 = 'std_ndre'
# feature1 = props[:, names.index(name1)]

# name2 = 'perc75_chm'
# feature2 = props[:, names.index(name2)]

# name3 = 'perc75_s'
# feature3 = props[:, names.index(name3)]

# x = feature1[positives]
# y = feature2[positives]
# z = feature3[positives]

# f, ax = plt.subplots(figsize=[4,3])
# _ = ax.scatter(
#     x,
#     y,
#     # c=z,
#     color='k',
#     s=5,
#     alpha=0.5,
#     )

# ax.set_xlabel(name1)
# ax.set_ylabel(name2)